In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import torch
import numpy as np

In [4]:
torch.__version__

'2.0.0'

全結合層の定義


In [5]:
import torch.nn as nn

fully_connected_layer = nn.Linear(3, 2)
fully_connected_layer.weight


Parameter containing:
tensor([[ 0.0722, -0.0527,  0.0838],
        [ 0.3957, -0.1869, -0.1136]], requires_grad=True)

In [6]:
fully_connected_layer.bias

Parameter containing:
tensor([ 0.4593, -0.2142], requires_grad=True)

In [7]:
# seed value fixed for reproducibility
torch.manual_seed(1)

In [8]:
fully_connected_layer = nn.Linear(3, 2)

print("weight: ", fully_connected_layer.weight)
print("bias: ", fully_connected_layer.bias)

weight:  Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119],
        [ 0.2710, -0.5435,  0.3462]], requires_grad=True)
bias:  Parameter containing:
tensor([-0.1188,  0.2937], requires_grad=True)


In [9]:
x = torch.tensor([[1, 2, 3]], dtype=torch.float32) # float32 is the default (これにしないとエラーになる)

print("x: ", x)
print("x shape: ", x.shape)
print("x type: ", x.dtype)

x:  tensor([[1., 2., 3.]])
x shape:  torch.Size([1, 3])
x type:  torch.float32


In [10]:
# 線形変換
u = fully_connected_layer(x)

In [11]:
u

tensor([[-0.6667,  0.5164]], grad_fn=<AddmmBackward0>)

非線形関数

PyTorch で使用される関数はすべて torch.nn.functions に含まれている。


In [12]:
import torch.nn.functional as F

# 活性化関数
# ReLU関数
h = F.relu(u)
h

tensor([[0.0000, 0.5164]], grad_fn=<ReluBackward0>)

In [13]:
# seed value fixed for reproducibility
torch.manual_seed(1)

# input values
x = torch.tensor([[1, 2, 3]], dtype=torch.float32)
x

tensor([[1., 2., 3.]])

In [14]:
# fully connected layer
fc1 = nn.Linear(3, 2)
fc2 = nn.Linear(2, 1)

# 線形変換
u1 = fc1(x)
u1

tensor([[-0.6667,  0.5164]], grad_fn=<AddmmBackward0>)

In [15]:
# 非線形変換
h1 = F.relu(u1)

In [16]:
# 線形変換
y = fc2(h1)
y

tensor([[0.1514]], grad_fn=<AddmmBackward0>)

### 損失関数

今回は、損失関数として、二乗誤差を使用する。


In [17]:
# 目標値
t = torch.tensor([[1]], dtype=torch.float32)
t

tensor([[1.]])

In [18]:
# MSE: Mean Squared Error
loss = F.mse_loss(y, t)
loss

tensor(0.7201, grad_fn=<MseLossBackward0>)

### 学習の一連の流れ

- Step1: データの準備
- Step2: ネットワークの定義
- Step3: 損失関数の定義
- Step4: 最適化手法の定義
- Step5: 学習の実行


Scikit-learn のアヤメデータセットを使用する。


In [19]:
from sklearn.datasets import load_iris

# Irisデータセットの読み込み
x, t = load_iris(return_X_y=True)

# データの確認
x.shape, t.shape # type: ignore

((150, 4), (150,))

In [20]:
# check the first 5 rows of the data
x[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [21]:
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [22]:
x.dtype, t.dtype # type: ignore

(dtype('float64'), dtype('int64'))

In [23]:
# データ型の変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

# check the data type
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [24]:
# データ型の確認
x.dtype, t.dtype

(torch.float32, torch.int64)

In [25]:
from torch.utils.data import TensorDataset

# 入力値と目標値を結合
dataset = TensorDataset(x, t)
dataset

In [26]:
type(dataset)

torch.utils.data.dataset.TensorDataset

In [27]:
# (入力変数, 教師データ)の形でデータが格納されている
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [28]:
type(dataset[0])

tuple

In [29]:
# 1サンプル目の入力変数
dataset[0][0]

tensor([5.1000, 3.5000, 1.4000, 0.2000])

In [30]:
# 1サンプル目の目標値
dataset[0][1]

tensor(0)

In [31]:
# サイズ数の確認
len(dataset)

150

In [32]:
# 各データセットのサンプル数を決定
n_train = int(len(dataset) * 0.6)
n_valid = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_valid

# それぞれのサンプル数を確認
n_train, n_valid, n_test

(90, 30, 30)

In [33]:
from torch.utils.data import random_split

torch.manual_seed(0)

train, valid, test = random_split(dataset, [n_train, n_valid, n_test])

# 各データセットのサンプル数を確認
len(train), len(valid), len(test)

(90, 30, 30)

ミニバッチ学習を行う場合には、各データセットからバッチサイズ分のサンプルを取得する必要があります。
このサンプル抽出の際、学習時にはランダムにシャッフルして抽出するなどの工夫があり、自前で実装することもできるが、Pytorch では torch.utils.data.DataLoader が用意されているので、これを使用する。


In [34]:
batch_size: int = 10

from torch.utils.data import DataLoader

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(x)
        x = self.fc2(x)
        return x

torch.manual_seed(0)

net = Net()

net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

損失関数を選択する。
今回はクロスエントロピーを採用する


In [36]:
criterion = nn.CrossEntropyLoss()
criterion

CrossEntropyLoss()

最適化手法を選択する。


In [37]:
# optimizerという名前で設定することが多い
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [38]:
batch = next(iter(train_loader))
batch

[tensor([[5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [6.5000, 3.0000, 5.5000, 1.8000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [6.3000, 2.5000, 4.9000, 1.5000],
         [7.1000, 3.0000, 5.9000, 2.1000],
         [5.8000, 2.7000, 4.1000, 1.0000],
         [7.0000, 3.2000, 4.7000, 1.4000],
         [6.7000, 3.0000, 5.0000, 1.7000],
         [7.2000, 3.6000, 6.1000, 2.5000]]),
 tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])]

In [39]:
x, t = batch

x, t

(tensor([[5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [6.5000, 3.0000, 5.5000, 1.8000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [6.3000, 2.5000, 4.9000, 1.5000],
         [7.1000, 3.0000, 5.9000, 2.1000],
         [5.8000, 2.7000, 4.1000, 1.0000],
         [7.0000, 3.2000, 4.7000, 1.4000],
         [6.7000, 3.0000, 5.0000, 1.7000],
         [7.2000, 3.6000, 6.1000, 2.5000]]),
 tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2]))

In [40]:
# 現状のパラメータ値
net.fc1.weight

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [41]:
net.fc1.bias

Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)

In [42]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [43]:
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [44]:
x = x.to(device)
x

tensor([[5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [6.3000, 2.5000, 4.9000, 1.5000],
        [7.1000, 3.0000, 5.9000, 2.1000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [7.0000, 3.2000, 4.7000, 1.4000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [7.2000, 3.6000, 6.1000, 2.5000]])

In [45]:
t = t.to(device)
t

tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])

In [46]:
max_epochs = 10

torch.manual_seed(0)

net = Net().to(device)

optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

for epoch in range(max_epochs):
    for batch in train_loader:
        # バッチサイズ分のサンプルを取得
        x, t = batch

        # 学習に使用するデバイスにデータを送る
        x = x.to(device)
        t = t.to(device)

        # 勾配の初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)

        # 損失の計算
        loss = criterion(y, t)

        print(f"epoch: {epoch}, loss: {loss.item()}")

        # 逆伝播
        loss.backward()

        # パラメータの更新
        optimizer.step()

epoch: 0, loss: 1.0881630182266235
epoch: 0, loss: 1.0393922328948975
epoch: 0, loss: 1.0028115510940552
epoch: 0, loss: 1.0250868797302246
epoch: 0, loss: 1.0088638067245483
epoch: 0, loss: 0.9351975321769714
epoch: 0, loss: 0.8939588665962219
epoch: 0, loss: 0.9765418171882629
epoch: 0, loss: 0.9651517868041992
epoch: 1, loss: 0.8108385801315308
epoch: 1, loss: 1.0551203489303589
epoch: 1, loss: 0.9642282724380493
epoch: 1, loss: 0.9662469625473022
epoch: 1, loss: 0.9328590631484985
epoch: 1, loss: 0.7749171257019043
epoch: 1, loss: 0.8114358186721802
epoch: 1, loss: 0.722326397895813
epoch: 1, loss: 0.9744229316711426
epoch: 2, loss: 0.8967071771621704
epoch: 2, loss: 0.6616588234901428
epoch: 2, loss: 0.7619713544845581
epoch: 2, loss: 0.7153902649879456
epoch: 2, loss: 0.6994240283966064
epoch: 2, loss: 0.8678095936775208
epoch: 2, loss: 0.9200589060783386
epoch: 2, loss: 0.7730971574783325
epoch: 2, loss: 0.8362565040588379
epoch: 3, loss: 0.6122375726699829
epoch: 3, loss: 0.735

In [47]:
y_label = torch.argmax(y, dim=1)
y_label

tensor([2, 2, 0, 1, 2, 2, 2, 2, 2, 0])

In [48]:
t

tensor([1, 2, 0, 1, 2, 1, 1, 2, 2, 0])

In [49]:
y_label == t

tensor([False,  True,  True,  True,  True, False, False,  True,  True,  True])

In [51]:
acc = (y_label == t).sum().item() / len(t)
acc

0.7

In [53]:
max_epochs = 10

torch.manual_seed(0)

net = Net().to(device)

optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

for epoch in range(max_epochs):
    for batch in train_loader:
        # バッチサイズ分のサンプルを取得
        x, t = batch

        # 学習に使用するデバイスにデータを送る
        x = x.to(device)
        t = t.to(device)

        # 勾配の初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)

        # 損失の計算
        loss = criterion(y, t)

        print(f"epoch: {epoch}, loss: {loss.item()}")
        y_label = torch.argmax(y, dim=1)
        acc = (y_label == t).sum().item() / len(t)
        print(f"epoch: {epoch}, acc: {acc}")

        # 逆伝播
        loss.backward()

        # パラメータの更新
        optimizer.step()

epoch: 0, loss: 1.0881630182266235
epoch: 0, acc: 0.6
epoch: 0, loss: 1.0393922328948975
epoch: 0, acc: 0.8
epoch: 0, loss: 1.0028115510940552
epoch: 0, acc: 0.6
epoch: 0, loss: 1.0250868797302246
epoch: 0, acc: 0.6
epoch: 0, loss: 1.0088638067245483
epoch: 0, acc: 0.6
epoch: 0, loss: 0.9351975321769714
epoch: 0, acc: 0.5
epoch: 0, loss: 0.8939588665962219
epoch: 0, acc: 0.4
epoch: 0, loss: 0.9765418171882629
epoch: 0, acc: 0.2
epoch: 0, loss: 0.9651517868041992
epoch: 0, acc: 0.4
epoch: 1, loss: 0.8108385801315308
epoch: 1, acc: 0.7
epoch: 1, loss: 1.0551203489303589
epoch: 1, acc: 0.2
epoch: 1, loss: 0.9642282724380493
epoch: 1, acc: 0.8
epoch: 1, loss: 0.9662469625473022
epoch: 1, acc: 0.5
epoch: 1, loss: 0.9328590631484985
epoch: 1, acc: 0.6
epoch: 1, loss: 0.7749171257019043
epoch: 1, acc: 0.6
epoch: 1, loss: 0.8114358186721802
epoch: 1, acc: 0.4
epoch: 1, loss: 0.722326397895813
epoch: 1, acc: 0.8
epoch: 1, loss: 0.9744229316711426
epoch: 1, acc: 0.2
epoch: 2, loss: 0.89670717716